# Notebook 1ου εργαστηρίου

Πριν ξεκινήσετε **εκτελέστε τα δύο επόμενα κελιά** για να φορτώσετε τις βιβλιοθήκες και τα εργαλεία της Chisel.

*(Σημ.: Αυτό θα πρέπει να γίνει μια φορά μόνο, στην αρχή. Αν όμως κάνετε kernel restart θα πρέπει να το επαναλάβετε!)*

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Μια γρήγορη εισαγωγή
Η Chisel είναι γλώσσα περιγραφής υλικού (hardware), με την οποία μπορείτε να περιγράψετε λογικά κυκλώματα. Στη συνέχεια, μπορείτε να ελέγξετε τα κυκλώματα που φτιάξατε ως προς την ορθή λειτουργία τους και να τα μετατρέψετε σε μορφή που μπορεί να υλοποιηθεί σε πραγματικό chip (FPGA ή ASIC).

Η Chisel είναι εξειδικευμένη γλώσσα πεδίου (domain specific language) και χρησιμοποιεί ως βάση τη γλώσσα γενικού σκοπού Scala. Η σύνταξη που θα χρησιμοποιήσουμε είναι της Scala, στο εργαστήριο θα δούμε μόνο τα στοιχεία σύνταξης που είναι απαραίτητα για τις ασκήσεις.

## Modules στην Chisel
`Module` είναι ένα αντικείμενο της Chisel που περιγράφει ένα κομμάτι κυκλώματος με τη λειτουργικότητά του. Για να είναι πλήρης η περιγραφή ενός `Module` θα πρέπει να δώσουμε τα εξής στοιχεία:
* Πόσα σήματα εισόδου και εξόδου έχει το module και τι εύρος (πόσα bits) είναι το κάθε σήμα.
* Ποιες είναι οι τιμές των σημάτων εξόδου για κάθε περίπτωση τιμών στις εισόδους και (ενδεχομένως) εσωτερικών τιμών κατάστασης (state).

### Παράδειγμα
Έστω ότι θέλουμε ένα module με:
* μία είσοδο `in` εύρους 2 bits
* μία έξοδο `out` με εύρος 1 bit και μία έξοδο `c` με εύρος 3 bits
* η έξοδος `out` θα πρέπει να ισούται (να είναι δηλαδή συνδεμένη) με το bit 0 της εισόδου `in`.
* η έξοδος `c` θα πρέπει να έχει πάντα την τιμή 5.

Το ζητούμενο module μπορεί να περιγραφεί στην Chisel ως εξής (διαβάστε στη συνέχεια την εξήγηση):

In [ ]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(2.W))
    val out = Output(UInt(1.W))
    val c = Output(UInt(3.W))  
  })
    
  io.out := io.in(0)
  io.c := 5.U
}

Παρατηρήστε ότι όλος ο κώδικας του module περιβάλλεται από το:

~~~scala
class MyModule extends Module {
  // εδώ βάζουμε όλον τον κώδικα του module
}
~~~

Το πρώτο πράγμα που βάζουμε στην περιγραφή του module είναι το αντικείμενο `io` με τα σήματα εισόδου και εξόδου:

~~~scala
  val io = IO(new Bundle {
    val in = Input(UInt(2.W))
    val out = Output(UInt(1.W))
    val c = Output(UInt(3.W))  
  })
~~~

όπου `val in = Input(UInt(2.W))` ορίζει ένα σήμα εισόδου με όνομα `in` τύπου UInt (αριθμός χωρίς πρόσημο) και εύρος 2 bits. Αντίστοιχα, το `val out = Output(UInt(1.W))` ορίζει ένα σήμα εξόδου με όνομα `out` τύπου UInt (αριθμός χωρίς πρόσημο) και εύρος 1 bit κ.ο.κ.

Μετά την περιγραφή των εισόδων και εξόδων ακολουθεί η περιγραφή της λειτουργικότητας:

~~~scala
  io.out := io.in(0)
  io.c := 5.U
~~~

**Προσοχή στο `:=`!** Ο τελεστής αυτός σημαίνει **«συνδέεται με»**. Π.χ. το `x := y` σημαίνει «το x συνδέεται και παίρνει τιμές (οδηγείται) από το y».

Στον προηγούμενο κώδικα λέμε ότι:
* η έξοδος `out` συνδέεται και παίρνει τιμή από το bit 0 της εισόδου `in` (η έκφραση `in(0)` επιλέγει το bit 0 από τα 2 bits που διαθέτει το `in`)
* η έξοδος `c` συνδέεται και παίρνει σταθερή τιμή 5.

Επειδή όλα τα σήματα δηλώνονται μέσα στο bundle `io`, παρατηρήστε ότι λέμε π.χ. `io.out` κι όχι απλά `out`.

## Σταθερές (literals) στην Chisel
Η Scala (άρα και η Chisel) είναι αυστηρή στη χρήση σταθερών ανάλογα με τον τύπο των δεδομένων. Στο προγούμενο παράδειγμα είδαμε δύο τύπους σταθερών:
* Σταθερές που δηλώνουν *εύρος* σημάτων (πόσα bits είναι το σήμα), π.χ. `2.W`. θα χρησιμοποιήσετε το `.W` όπου απαιτείται σταθερά εύρους.
* Σταθερές που δηλώνουν *τιμές* σημάτων (τι τιμές παίρνουν τα bits ενός σήματος). Αυτές οι σταθερές εξαρτώνται από τον τύπο του σήματος και, για τους αριθμούς χωρίς πρόσημο (UInts) που έχουμε εδώ, χρησιμοποιούν το `.U`, π.χ. `5.U`.
  * Μπορούμε να γράψουμε την τιμή ενός σήματος δεκαδικά (όπως το `5.U`) ή δυαδικά (π.χ. το `5.U` μπορεί να γραφεί και ως `"b101".U`).
  
## Έλεγχος ορθότητας
Μετά την περιγραφή του κυκλώματός μας σε ένα module πρέπει να ελέγξουμε αν λειτουργεί σωστά. Η Chisel μας παρέχει τη συνάρτηση `test`, στο περιβάλλον της οποίας χρησιμοποιούμε τις μεθόδους `poke()` και `expect()`:
* Πρώτα θέτουμε τιμές στα σήματα εισόδου μέσω της μεθόδου τους `poke(νέα-τιμή-σήματος)`.
* Στη συνέχεια ελέγχουμε αν οι έξοδοι έχουν την αναμενόμενη τιμή με τη μέθοδο `expect(αναμενόμενη-τιμή)`. Αν η έξοδος δεν έχει την αναμενόμενη τιμή προκαλείται σφάλμα.

### Παράδειγμα ελέγχου
Στη συνέχεια ακολουθεί παράδειγμα ελέγχου για το module που ορίσαμε προηγουμένως. Σε κανονικές συνθήκες θα πρέπει να ελεγχθούν εξαντλητικά οι έξοδοι για όλες οι τιμές εισόδου, για απλότητα εδώ γίνονται μερικοί έλεγχοι μόνο.

In [ ]:
test(new MyModule()) { c =>
  // βάζουμε 00 στο in
  c.io.in.poke(0.U)
  // το out θα πρέπει να είναι 0  
  c.io.out.expect(0.U)

  // βάζουμε 01 στο in
  c.io.in.poke("b01".U)
  // το out θα πρέπει να είναι 1  
  c.io.out.expect(1.U)

  // το c θα πρέπει να είναι πάντα 5
  c.io.c.expect(5.U)  
}
println("SUCCESS!!")

Παρατηρήστε ότι στο προηγούμενο παράδειγμα η μεταβλητή `c` συμβολίζει το module που ελέγχεται, συνεπώς για να συμβολίσουμε π.χ. την είσοδο `in` θα πρέπει να γράψουμε `c.io.in`.

## Δοκιμάστε κι εσείς
Συμπληρώστε το επόμενο κελί για να ορίσετε το module `MyModule2` το οποίο:
* Θα έχει δύο εισόδους `a` και `b` των 2 bits η καθεμία
* Θα έχει δύο εξόδους `out1` και `out2` του 1 bit η καθεμία
* H έξοδος `out1` θα συνδέεται με το bit 1 της εισόδου `a`
* H έξοδος `out2` θα συνδέεται με το bit 0 της εισόδου `b`

In [ ]:
class MyModule2 extends Module {
  val io = IO(new Bundle {
    // ορίστε τα σήματα εισόδου και εξόδου εδώ
  })
    
  // προσδιορίστε τις τιμές που παίρνουν τα out1, out2 εδώ
}

Στη συνέχεια προσθέστε τους εξής ελέγχους στο επόμενο κελί:
* Θέστε το `a` στην τιμή 2. Το `out1` θα πρέπει να είναι 1.
* Θέστε το `b` στην τιμή 1. Το `out2` θα πρέπει να είναι 1.
* Θέστε το `a` στην τιμή 1. Το `out1` θα πρέπει να είναι 0.
* Θέστε το `b` στην τιμή 2. Το `out2` θα πρέπει να είναι 0.

In [ ]:
test(new MyModule2()) { c =>

  // προσθέστε τους ελέγχους σας με poke() και expect() εδώ
    
}
println("SUCCESS!!")

## Τελειώνοντας...
Μπορείτε να κατεβάσετε και να πάρετε μαζί σας το σημερινό notebook για να το έχετε στο αρχείο σας.

**Δεν ανήκει στα παραδοτέα του εργαστηρίου.**